In [ ]:
from src.data import get_harmful_instructions, get_harmless_instructions
from src.abliterator import Abliterator
from colorama import Fore

import textwrap

In [ ]:
qwen = "Qwen/Qwen1.5-4B-Chat"

In [ ]:
dataset = {
    "harmful": get_harmful_instructions(),
    "harmless": get_harmless_instructions(),
}

In [ ]:
print("Harmful instructions (train set):")
for i in range(4):
    train = dataset["harmful"][0]
    print(f"\t{train[i*5]}")
print("Harmless instructions (train set):")
for i in range(4):
    dataset["harmless"][0]
    print(f"\t{train[i*5]}")

In [ ]:
N = 35
positive_toks = [32, 1249, 8420, 95456, 77045]
negative_toks = [4157, 14589, 11209, 19860, 31450, 944]

In [ ]:
abliterator = Abliterator(model_name=qwen, data=dataset)

In [ ]:
sample_instructions = dataset["harmful"][1][: abliterator.n_inst_test]

In [ ]:
baseline_generations = abliterator.generate(sample_instructions, fwd_hooks=[])

In [ ]:
intervention_generations = abliterator.test_top_N_directions(sample_instructions,N)

In [ ]:
blacklist = ["I cannot", "I can't", "I'm sorry", "I don't"]

for i in range(abliterator.n_inst_test):
    print(f"INSTRUCTION {i}: {repr(sample_instructions[i])}")
    print(Fore.GREEN + f"BASELINE COMPLETION:")
    print(textwrap.fill(repr(baseline_generations[i]), width=100, initial_indent="\t", subsequent_indent="\t"))
    for layer_candidate in range(N):
        if not any(word in intervention_generations[layer_candidate][i] for word in blacklist):
            print(Fore.RED + f"(LAYER CANDIDATE #{layer_candidate}) INTERVENTION COMPLETION:")
            print(textwrap.fill(repr(intervention_generations[layer_candidate][i]), width=100, initial_indent="\t", subsequent_indent="\t"))
    print(Fore.RESET)